Goal: Load 3-5 images for the 10 most-likely species near a given location.


In [8]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
locations_of_interest = ["Providence, Rhode Island, United States"]

In [ ]:
# Load geocodes for the locations of interest
from birding.geocoding import retrieve_geocode
from birding.primitives import Coordinate
from birding.sqlite_cache import init_db

init_db()
raw_location_data = {loc: retrieve_geocode(query=loc) for loc in locations_of_interest}
location_data = {loc: data for loc, data in raw_location_data.items() if data is not None}
assert len(raw_location_data) == len(location_data), "A location's geocode data was None!"

location_coords = {loc: Coordinate.from_geocode_data(data) for loc, data in location_data.items()}
location_coords

Geocode for query 'Providence, Rhode Island, United States' was already cached.


{'Providence, Rhode Island, United States': Coordinate(latitude=41.8239891, longitude=-71.4128343)}

In [5]:
# Load hotspots near each location of interest
from geopy.distance import geodesic

from birding.ebird_api import get_ebird_api_key, retrieve_nearby_hotspots
from birding.primitives import EBirdHotspot

ebird_api_key = get_ebird_api_key()
hotspots_data_per_loc = {
    loc: retrieve_nearby_hotspots(ebird_api_key, coord) for loc, coord in location_coords.items()
}
hotspots_per_loc = {
    loc: {EBirdHotspot.from_json(hotspot_data) for hotspot_data in hotspots_data}
    for loc, hotspots_data in hotspots_data_per_loc.items()
}

# Find the nearest hotspot to each location
nearest_hotspots: dict[str, EBirdHotspot] = {}
for loc, hotspots in hotspots_per_loc.items():
    loc_coord = location_coords[loc]
    nearest = min(hotspots, key=lambda hs: geodesic(loc_coord, hs.location.coord).mi)
    nearest_hotspots[loc] = nearest

nearest_hotspots

Nearby hotspots for (41.8240, -71.4128) were already cached.


{'Providence, Rhode Island, United States': EBirdHotspot(location=EBirdLocation(id='L24184797', name='Kennedy Plaza, Providence', coord=Coordinate(latitude=41.825473, longitude=-71.412127), country_code='US', subnat1_code='US-RI', subnat1_name=None, subnat2_code='US-RI-007', subnat2_name=None), all_time_species=38)}

In [6]:
# Retrieve species lists for surrounding subnational and national areas
from birding.ebird_api import retrieve_species_list

all_species_lists: dict[str, list[str]] = {}
invalid_area_codes = {}

for loc, hotspots in hotspots_per_loc.items():
    print(f"Retrieving species lists for subnational regions around '{loc}'...")
    for hotspot in hotspots:
        hs_loc = hotspot.location
        for possible_area_code in {hs_loc.subnat1_code, hs_loc.subnat2_code, hs_loc.country_code}:
            if possible_area_code is None or possible_area_code in invalid_area_codes:
                continue

            new_list = retrieve_species_list(ebird_api_key, possible_area_code)
            all_species_lists[possible_area_code] = new_list

species_union = set()
for species_list in all_species_lists.values():
    species_union.update(species_list)

print(f"Current species lists include {len(species_union)} total distinct species.")

Retrieving species lists for subnational regions around 'Providence, Rhode Island, United States'...
Bird species list for 'US' was already cached.
Bird species list for 'US-RI' was already cached.
Bird species list for 'US-RI-007' was already cached.
Bird species list for 'US' was already cached.
Bird species list for 'US-RI' was already cached.
Bird species list for 'US-RI-007' was already cached.
Bird species list for 'US' was already cached.
Bird species list for 'US-RI' was already cached.
Bird species list for 'US-RI-007' was already cached.
Bird species list for 'US' was already cached.
Bird species list for 'US-RI' was already cached.
Bird species list for 'US-RI-007' was already cached.
Bird species list for 'US-MA' was already cached.
Bird species list for 'US-MA-021' was already cached.
Bird species list for 'US' was already cached.
Bird species list for 'US' was already cached.
Bird species list for 'US-RI' was already cached.
Bird species list for 'US-RI-007' was already c

In [ ]:
# Finally, retrieve species lists for all hotspots near our locations of interest
for loc, hotspots in hotspots_per_loc.items():
    print(f"Retrieving species lists for hotspots near '{loc}'...")
    for hotspot in hotspots:
        hotspot_species = retrieve_species_list(ebird_api_key, hotspot.location.id)
        all_species_lists[hotspot.location.id] = hotspot_species

species_union = set()
for species_list in all_species_lists.values():
    species_union.update(species_list)

print(f"Current species lists include {len(species_union)} total distinct species.")